# TensorFlow Serving

当我们将模型训练完毕后，往往需要将模型在生产环境中部署。最常见的方式，是在服务器上提供一个 API，即客户机向服务器的某个 API 发送特定格式的请求，服务器收到请求数据后通过模型进行计算，并返回结果。如果仅仅是做一个 Demo，不考虑高并发和性能问题，其实配合 [Flask](https://palletsprojects.com/p/flask/) 等 Python 下的 Web 框架就能非常轻松地实现服务器 API。不过，如果是在真的实际生产环境中部署，这样的方式就显得力不从心了。这时，TensorFlow 为我们提供了 TensorFlow Serving 这一组件，能够帮助我们在实际生产环境中灵活且高性能地部署机器学习模型。

## TensorFlow Serving 安装 

TensorFlow Serving 可以使用 apt-get 或 Docker 安装。在生产环境中，推荐 [使用 Docker 部署 TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker) 。

```
# Download the TensorFlow Serving Docker image and repo
docker pull tensorflow/serving

git clone https://github.com/tensorflow/serving
# Location of demo models
TESTDATA="$(pwd)/serving/tensorflow_serving/servables/tensorflow/testdata"

# Start TensorFlow Serving container and open the REST API port
docker run -t --rm -p 8501:8501 \
    -v "$TESTDATA/saved_model_half_plus_two_cpu:/models/half_plus_two" \
    -e MODEL_NAME=half_plus_two \
    tensorflow/serving &

# Query the model using the predict API
curl -d '{"instances": [1.0, 2.0, 5.0]}' \
    -X POST http://localhost:8501/v1/models/half_plus_two:predict

# Returns => { "predictions": [2.5, 3.0, 4.5] }
```

## Serving with Docker

### Pulling a serving image

Once you have Docker installed, you can pull the latest TensorFlow Serving docker image by running:

```shell
docker pull tensorflow/serving
```

This will pull down an minimal Docker image with TensorFlow Serving installed.

See the Docker Hub [tensorflow/serving repo](http://hub.docker.com/r/tensorflow/serving/tags/) for other versions of images you can pull.

## Running a serving image
### The serving images (both CPU and GPU) have the following properties:

Port 8500 exposed for gRPC
Port 8501 exposed for the REST API
Optional environment variable MODEL_NAME (defaults to model)
Optional environment variable MODEL_BASE_PATH (defaults to /models)
When the serving image runs ModelServer, it runs it as follows:

```shell
tensorflow_model_server --port=8500 --rest_api_port=8501 \
  --model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME}
```

To serve with Docker, you'll need:

- An open port on your host to serve on
- A SavedModel to serve
- A name for your model that your client will refer to

What you'll do is [run the Docker](https://docs.docker.com/engine/reference/run/) container, [publish](https://docs.docker.com/engine/reference/commandline/run/#publish-or-expose-port--p---expose) the container's ports to your host's ports, and mounting your host's path to the SavedModel to where the container expects models.

Let's look at an example:

```shell
docker run -p 8501:8501 \
  --mount type=bind,source=/path/to/my_model/,target=/models/my_model \
  -e MODEL_NAME=my_model -t tensorflow/serving
```


在这种情况下，我们启动了一个Docker容器，将REST API端口8501发布到主机的端口8501，并获取了一个我们命名的模型`my_model`并将其绑定到默认的模型基本路径（`${MODEL_BASE_PATH}/${MODEL_NAME}`= `/models/my_model`）。最后，我们填补了环境变量 `MODEL_NAME`有`my_model`，离开`MODEL_BASE_PATH`它的默认值。

这将在容器中运行：

```shell
tensorflow_model_server --port=8500 --rest_api_port=8501 \
  --model_name=my_model --model_base_path=/models/my_model
```

如果要发布gRPC端口，可以使用`-p 8500:8500`。您可以同时打开gRPC和REST API端口，或者选择仅打开一个端口。

传递其他参数

`tensorflow_model_server`支持许多其他参数，您可以将这些参数传递给服务的Docker容器。例如，如果我们要传递模型配置文件而不是指定模型名称，则可以执行以下操作：

```shell
docker run -p 8500:8500 -p 8501:8501 \
  --mount type=bind,source=/path/to/my_model/,target=/models/my_model \
  --mount type=bind,source=/path/to/my/models.config,target=/models/models.config \
  -t tensorflow/serving --model_config_file=/models/models.config
```

此方法适用于支持的任何其他命令行参数 `tensorflow_model_server`。

创建自己的服务图片

如果您想要将模型内置到容器中的服务图像，则可以创建自己的图像。

首先将服务图片作为守护程序运行：

```shell
docker run -d --name serving_base tensorflow/serving
```

接下来，将SavedModel复制到容器的model文件夹中：

```shell
docker cp models/<my model> serving_base:/models/<my model>
```

最后，通过更改`MODEL_NAME` 以匹配模型名称`来提交为模型提供服务的容器'：

```shell
docker commit --change "ENV MODEL_NAME <my model>" serving_base <my container>
```

现在可以停止 `serving_base`

```shell
docker kill serving_base
```

这将为您提供一个名为Docker的映像``，您可以对其进行部署并加载模型以在启动时提供服务。

服务示例

让我们来看一个完整的示例，在该示例中加载SavedModel并使用REST API对其进行调用。首先拉动投放图片：

```shell
docker pull tensorflow/serving
```

这将拉出安装了ModelServer的最新TensorFlow Serving映像。

接下来，我们将使用一个名为的玩具模型`Half Plus Two`，该模型会`0.5 * x + 2`为`x`我们提供的预测值生成。

要获得此模型，请首先克隆TensorFlow服务回购。

```
mkdir -p /tmp/tfserving
cd /tmp/tfserving
git clone https://github.com/tensorflow/serving
```

Next, run the TensorFlow Serving container pointing it to this model and opening the REST API port (8501):

```shell
docker run -p 8501:8501 --mount type=bind,source=/tmp/tfserving/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu,target=/models/half_plus_two -e MODEL_NAME=half_plus_two -t tensorflow/serving &
```

This will run the docker container and launch the TensorFlow Serving Model Server, bind the REST API port 8501, and map our desired model from our host to where models are expected in the container. We also pass the name of the model as an environment variable, which will be important when we query the model.

To query the model using the predict API, you can run

```shell
curl -d '{"instances": [1.0, 2.0, 5.0]}' \
  -X POST http://localhost:8501/v1/models/half_plus_two:predict
```

NOTE: Older versions of Windows and other systems without curl can download it [here](https://curl.haxx.se/download.html).

This should return a set of values:

```json
{ "predictions": [2.5, 3.0, 4.5] }
```

More information on using the RESTful API can be found [here](https://www.tensorflow.org/tfx/serving/api_rest).

#### TensorFlow Serving 可以直接读取 SavedModel 格式的模型进行部署（导出模型到 SavedModel 文件的方法见 [前文](https://tf.wiki/zh/deployment/export.html#savedmodel) ）。使用以下命令即可：

```
tensorflow_model_server \
    --rest_api_port=端口号（如8501） \
    --model_name=模型名 \
    --model_base_path="SavedModel格式模型的文件夹绝对地址（不含版本号）"
```

注解

TensorFlow Serving 支持热更新模型，其典型的模型文件夹结构如下：

```
/saved_model_files
    /1      # 版本号为1的模型文件
        /assets
        /variables
        saved_model.pb
    ...
    /N      # 版本号为N的模型文件
        /assets
        /variables
        saved_model.pb
```

上面 1~N 的子文件夹代表不同版本号的模型。当指定 `--model_base_path` 时，只需要指定根目录的 **绝对地址** （不是相对地址）即可。例如，如果上述文件夹结构存放在 `home/snowkylin` 文件夹内，则 `--model_base_path` 应当设置为 `home/snowkylin/saved_model_files` （不附带模型版本号）。TensorFlow Serving 会自动选择版本号最大的模型进行载入。

### Keras Sequential 模式模型的部署 

由于 Sequential 模式的输入和输出都很固定，因此这种类型的模型很容易部署，无需其他额外操作。例如，要将 [前文使用 SavedModel 导出的 MNIST 手写体识别模型](https://tf.wiki/zh/deployment/export.html#savedmodel) （使用 Keras Sequential 模式建立）以 `MLP` 的模型名在 `8501` 端口进行部署，可以直接使用以下命令：

```
tensorflow_model_server \
    --rest_api_port=8501 \
    --model_name=MLP \
    --model_base_path="/home/.../.../saved"  # 文件夹绝对地址根据自身情况填写，无需加入版本号
```

然后就可以按照 [后文的介绍](https://tf.wiki/zh/deployment/serving.html#call-serving-api) ，使用 gRPC 或者 RESTful API 在客户端调用模型了。

### 自定义 Keras 模型的部署 

使用继承 `tf.keras.Model` 类建立的自定义 Keras 模型的自由度相对更高。因此当使用 TensorFlow Serving 部署模型时，对导出的 SavedModel 文件也有更多的要求：

- 需要导出到 SavedModel 格式的方法（比如 `call` ）不仅需要使用 `@tf.function` 修饰，还要在修饰时指定 `input_signature` 参数，以显式说明输入的形状。该参数传入一个由 `tf.TensorSpec` 组成的列表，指定每个输入张量的形状和类型。例如，对于 MNIST 手写体数字识别，我们的输入是一个 `[None, 28, 28, 1]` 的四维张量（ `None` 表示第一维即 Batch Size 的大小不固定），此时我们可以将模型的 `call` 方法做以下修饰：

```
class MLP(tf.keras.Model):
    ...

    @tf.function(input_signature=[tf.TensorSpec([None, 28, 28, 1], tf.float32)])
    def call(self, inputs):
        ...
```

- 在将模型使用 `tf.saved_model.save` 导出时，需要通过 `signature` 参数提供待导出的函数的签名（Signature）。简单说来，由于自定义的模型类里可能有多个方法都需要导出，因此，需要告诉 TensorFlow Serving 每个方法在被客户端调用时分别叫做什么名字。例如，如果我们希望客户端在调用模型时使用 `call` 这一签名来调用 `model.call` 方法时，我们可以在导出时传入 `signature` 参数，以 `dict` 的键值对形式告知导出的方法对应的签名，代码如下：

```
model = MLP()
...
tf.saved_model.save(model, "saved_with_signature/1", signatures={"call": model.call})
```

以上两步均完成后，即可使用以下命令部署：

```
tensorflow_model_server \
    --rest_api_port=8501 \
    --model_name=MLP \
    --model_base_path="/home/.../.../saved_with_signature"  # 修改为自己模型的绝对地址
```

## 在客户端调用以 TensorFlow Serving 部署的模型 

TensorFlow Serving 支持以 gRPC 和 RESTful API 调用以 TensorFlow Serving 部署的模型。本手册主要介绍较为通用的 RESTful API 方法。

RESTful API 以标准的 HTTP POST 方法进行交互，请求和回复均为 JSON 对象。为了调用服务器端的模型，我们在客户端向服务器发送以下格式的请求：

服务器 URI： `http://服务器地址:端口号/v1/models/模型名:predict`

请求内容：

```
{
    "signature_name": "需要调用的函数签名（Sequential模式不需要）",
    "instances": 输入数据
}
```

回复为：

```
{
    "predictions": 返回值
}
```

### Python 客户端示例 

以下示例使用 [Python 的 Requests 库](https://2.python-requests.org//zh_CN/latest/user/quickstart.html) （你可能需要使用 `pip install requests` 安装该库）向本机的 TensorFlow Serving 服务器发送 MNIST 测试集的前 10 幅图像并返回预测结果，同时与测试集的真实标签进行比较。

```
import json
import numpy as np
import requests
from zh.model.utils import MNISTLoader


data_loader = MNISTLoader()
data = json.dumps({
    "instances": data_loader.test_data[0:3].tolist()
    })
headers = {"content-type": "application/json"}
json_response = requests.post(
    'http://localhost:8501/v1/models/MLP:predict',
    data=data, headers=headers)
predictions = np.array(json.loads(json_response.text)['predictions'])
print(np.argmax(predictions, axis=-1))
print(data_loader.test_label[0:10])
```

输出：

```
[7 2 1 0 4 1 4 9 6 9]
[7 2 1 0 4 1 4 9 5 9]
```

可见预测结果与真实标签值非常接近。

对于自定义的 Keras 模型，在发送的数据中加入 `signature_name` 键值即可，即将上面代码的 `data` 建立过程改为

```
data = json.dumps({
    "signature_name": "call",
    "instances": data_loader.test_data[0:10].tolist()
    })
```

Reference:
    https://tf.wiki/zh/deployment/serving.html
        https://www.tensorflow.org/tfx/serving/docker